#  Covid-19 Dashboard

This is a DIY Dashboard for Coronavirus data. It shows a comparison of the number of new cases, carried out tests and planned testing capacity.

Data Source: *Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).*


In [5]:
from uk_covid19 import Cov19API
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as wdg
import json

In [6]:
%matplotlib inline
plt.rcParams['figure.dpi'] = 100

In [7]:
# Load JSON files and store the raw data in some variable. Edit as appropriate
#jsondata={}
with open("testsituation.json", "rt") as INFILE:
    data=json.load(INFILE)

In [12]:
def parseDate(datestring):
    """ Convert a date string into a pandas datetime object """
    return pd.to_datetime(datestring, format="%Y-%m-%d")

def wrangle_data(rawdata):
    datalist=rawdata['data']
    dates=[dictionary['date'] for dictionary in datalist ]
    dates.sort()
    startDate=parseDate(dates[0])
    endDate=parseDate(dates[-1])
    index=pd.date_range(startDate, endDate, freq='D')
    testsituationdf=pd.DataFrame(index=index, columns=['cases', 'test', 'capacity'])
    for entry in datalist: # each entry is a dictionary with date, cases, test and capacity
        date=parseDate(entry['date'])
        for column in ['cases', 'test', 'capacity']:
            if pd.isna(testsituationdf.loc[date, column]): 
                value= float(entry[column]) if entry[column]!=None else 0.0
                testsituationdf.loc[date, column]=value
    testsituationdf.fillna(0.0, inplace=True)            
    #testsituationdf.plot() 
    #testsituationdf.plot(logy=True)
    return testsituationdf
df=wrangle_data(data)

## UK Summary

The latest daily new cases is estimated at 18000 with a steady daily infection rate as of 20 November 2020.
The line chart can be diplayed in linear scale or log scale.

In [13]:
# Place your API access code in this function. Do not call this function directly; it will be called by 
# the button callback. 
def access_api():
    filters = [
        'areaType=overview' # note each metric-value pair is inside one string
    ]
    structure = {
        "date": "date",
        "cases": "newCasesByPublishDate",
        "test": "newTestsByPublishDate",
        "capacity": "plannedCapacityByPublishDate"
    }
    api = Cov19API(filters=filters, structure=structure)
    testsituation=api.get_json()
    return testsituation # return data read from the API

In [14]:
#Graphs and Analysis
stats=wdg.SelectMultiple(
    options=['cases', 'test', 'capacity'],
    value=['cases', 'test', 'capacity'],
    rows=3,
    description='Stats:',
    disabled=False
)

scale=wdg.RadioButtons(
    options=['linear', 'log'],
    description='Scale:',
    disabled=False
)

controls=wdg.VBox([stats, scale])

def testsituation_graph(gcols, gscale):
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        df[list(gcols)].plot(logy=logscale)
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")

graph=wdg.interactive_output(testsituation_graph, {'gcols': stats, 'gscale': scale})
display(controls, graph)

def refresh_graph():
    """ We change the value of the widget in order to force a redraw of the graph;
    Here I use the scale widget to enable the redraw """
    current=scale.value
    if current==scale.options[0]:
        other=scale.options[1]
    else:
        other=scale.options[0]
    scale.value=other # forces the redraw
    scale.value=current # now we can change it back 

# Printout from this function will be lost in Voila unless captured in an
# output widget - therefore, we give feedback to the user by changing the 
# appearance of the button
def api_button_callback(button):
    """ Button callback - it must take the button as its parameter (unused in this case).
    Accesses API, wrangles data, updates global variable df used for plotting. """
    # Get fresh data from the API. If you have time, include some error handling
    # around this call.
    apidata=access_api()
    # wrangle the data and overwrite the dataframe for plotting
    global df
    df=wrangle_data(apidata)
    # the graph won't refresh until the user interacts with the widget.
    # this function simulates the interaction, see Graph and Analysis below.
    # you can omit this step in the first instance
    refresh_graph()
    # after all is done, you can switch the icon on the button to a "check" sign
    # and optionally disable the button - it won't be needed again. You can use icons
    # "unlink" or "times" and change the button text to "Unavailable" in case the 
    # api call fails.
    apibutton.icon="check"
    # apibutton.disabled=True

    
apibutton=wdg.Button(
    description='Refresh data', # you may want to change this...
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Click to refresh data from Apood luck",
    # FontAwesome names without the `fa-` prefix - try "download"
    icon='exclamation-triangle'
)

# remember to register your button callback function with the button
apibutton.on_click(api_button_callback) # the name of your function inside these brackets

display(apibutton)

# run all cells before clicking on this button

Output()

Button(button_style='info', description='Refresh data', icon='exclamation-triangle', style=ButtonStyle(), tool…